In [1]:
import numpy as np
import thewalrus as tw
import shutil
import multipartite_state as ms
np.set_printoptions(linewidth=320, edgeitems=999)


In [28]:
BS = tw.symplectic.beam_splitter(np.pi/4,0)
print(BS)
tw.symplectic.expand(BS,[0,2,1,3,5,6,7,8],10)


[[ 0.70710678 -0.70710678 -0.         -0.        ]
 [ 0.70710678  0.70710678 -0.         -0.        ]
 [ 0.          0.          0.70710678 -0.70710678]
 [ 0.          0.          0.70710678  0.70710678]]


ValueError: shape mismatch: value array of shape (2,2) could not be broadcast to indexing result of shape (8,8)

In [52]:
N=100
F = np.eye(2*N)
A = np.ones((2*N,2*N)) *1/2

r = 0.6
modes = np.arange(4,8)

F[modes,modes] = np.exp(-2*r)
F[modes+N,modes+N] = np.exp(2*r)
F[modes+N,modes] =1
F[modes,modes+N] =1
F

array([[1.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.       

In [55]:
q = np.arange(4,8)
a = q + N
B = A.copy()
B[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))] = F[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))]@ A[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))] @ F[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))].conj().T

ar = np.arange(2*N)
mask_q = np.isin(ar, q)
mask_a = np.isin(ar, a)

# Combine masks to find elements not in q or a
mask = ~(mask_q | mask_a)
B[np.ix_(np.concatenate([q,a]),ar[mask])] = F[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))]@B[np.ix_(np.concatenate([q,a]),ar[mask])] 
B[np.ix_(ar[mask],np.concatenate([q,a]))] = B[np.ix_(ar[mask],np.concatenate([q,a]))]@ F[np.ix_(np.concatenate([q,a]),np.concatenate([q,a]))].conj().T
print(np.allclose(B,F@A@F.conj().T))



True


### Test implementation of single modes gates

In [14]:
cs = ms.cluster_state()
cs.state.cov = cs.state.cov +1
cs.squeeze_initial_state(r= 0.2)
A = cs.state.cov.copy()
cs.rotate_half_state(-np.pi/2)
cs.rotate_half_state(np.pi/2)
np.allclose(A,cs.state.cov)

True

#### Comparison with thewalrus entire calculation

In [38]:
n, m, k , N, ms, depth = cs.n, cs.m, cs.k, cs.N, cs.macronode_size, cs.spatial_depth
S = tw.symplectic.squeezing(0.2)

F = tw.symplectic.expand(S,list(np.arange(N*ms,N*ms*depth)),N*ms*depth)
C = np.eye(N*ms*depth*2)*1/2+1
np.allclose(F@C@F.conj().T,A)

True

The strategie used works and save time by applying calculation only on needed element of the cov matrix

---

### Test implementation of the BS among different modes

In [2]:
import multipartite_state as ms
cs = ms.cluster_state()
cs.state.cov +=1
cs.apply_beamsplitter(0) 
A = cs.state.cov
A

(256, 256)
(256, 256)


array([[ 5.00000000e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.11855716e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,

In [3]:
n, m, k , N, ms, depth = cs.n, cs.m, cs.k, cs.N, cs.macronode_size, cs.spatial_depth
indices = cs.BS_indices[0]
F = cs.BS(N = N*ms*depth,
                     modesA = indices[0],
                     modesB = indices[1])
BS = cs.BS(N = 2*len(indices[0]),
                modesA = np.arange(len(indices[0])),
                modesB = np.arange(len(indices[0])) + indices[1] - indices[0])

C = np.eye(N*ms*depth*2)*1/2+1
np.allclose(F@C@F.conj().T,A)


(256, 256)
(256, 256)


True

##### It does freaking Work!!!!!!!!!

In [16]:
A = np.ones((16,16))
a = np.array([2,5,0])
q = np.array([7,6,1])
indices = np.array([a,q])
print(indices)
indices = np.array(sorted(zip(a, q), key=lambda x: x[0])).T
print(indices +8)
F = cs.BS(N = 8,
                     modesA = indices[0],
                     modesB = indices[1])

print(np.arange(len(indices[0])) + indices[1] - indices[0])
BS = cs.BS(N = 2*len(indices[0]),
                modesA = np.arange(len(indices[0])),
                modesB = np.arange(len(indices[0]),2*len(indices[0])))
Q = A.copy()
A = F @ A @ F.conj().T
i = np.concatenate([np.concatenate(indices),np.concatenate(indices+8)])
ar = np.arange(2*8)
mask = ~np.isin(ar, i)


slices = np.ix_(i,i)
slices_B = np.ix_(i,ar[mask])
slices_C = np.ix_(ar[mask],i)
Q[slices] = BS @ Q[slices] @ BS.conj().T
Q[slices_B] = BS @ Q[slices_B] 
Q[slices_C] = Q[slices_C] @ BS.conj().T

np.isclose(A,Q)

[[2 5 0]
 [7 6 1]]
[[ 8 10 13]
 [ 9 15 14]]
(16, 16)
[1 6 3]
(12, 12)


array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  Tr